In [38]:
import numpy as np
import pandas as pd
import pickle

In [39]:
#dataFile=open('datasets/naive_bayes_data.txt','r+',encoding='utf-8')

In [40]:
def dataReader():
    with open('datasets/naive_bayes_data.txt','r+',encoding='utf-8') as file:
        dataFile=file.read()
    return dataFile

In [41]:
def processList(entries):
    lines=[]
    trgt=[]
    for entry in entries:        
        trgt.append(entry[:entry.find(' ')])
        value=entry[entry.find(' ')+1:]
        
        lines.append( (value[value.find("txt "):]).replace('txt ','') )
    return [lines, trgt]
        

In [42]:
def makeList(dataFile):
    lines=[]
    entries=dataFile.split('\n')
    for entry in entries:
        lines.append(entry[entry.find(' ')+1:])
    np.random.shuffle(lines)
    return lines

In [43]:
def testSplit(values):
    num_of_rows = int(len(values) * 0.8)
    train_data =values[:num_of_rows] 
    test_data = values[num_of_rows:]
    return train_data,test_data

In [44]:
def saveValues(lines):
    with open('datasets/data.pkl','wb') as file:
        pickle.dump(lines,file)

In [45]:
def makeVocab(lines,target):
    unwanted_chars = ".,-_ )!(?':;&/\*"
    poswordfreq = {}
    negwordfreq={}
    count=0;
    for words in lines: 
        words=words.split(' ')
        for raw_word in words:
            word = raw_word.strip(unwanted_chars)
            if target[count]=='pos':
                if word not in poswordfreq:
                    poswordfreq[word] = 0 
                poswordfreq[word] += 1
            else:
                if word not in negwordfreq:
                    negwordfreq[word] = 0 
                negwordfreq[word] += 1
        count=count+1
    del poswordfreq[""]
    del negwordfreq[""]    
    return poswordfreq,negwordfreq

In [46]:
def findProbability(wordfreq):
    length=sum(wordfreq.values())
    for key in wordfreq:
        wordfreq[key]=wordfreq.get(key)/length
    return wordfreq
    

In [47]:
def findclassProbability(target):
    poscount=target.count('pos')/len(target)
    negcount=target.count('neg')/len(target)
    return poscount,negcount

In [48]:
def makePrediction(test_lines,posfreq,negfreq,pprob,nprob):
    pred=[]
    for line in test_lines:
        line=line.split(' ')
        posprob=1
        negprob=1
        for word in line:
            if posfreq.get(word)!=None:
                posprob=posprob*posfreq.get(word)
            if negfreq.get(word)!=None:
                negprob=negprob*negfreq.get(word)
        posprob=posprob*pprob
        negprob=nprob*negprob
        if negprob>posprob:
            pred.append('neg')
        else:
            pred.append('pos')
    return pred

In [49]:
def findError(pred,trgt):
    error=0
    for i in range(len(trgt)):
         if pred[i]!=trgt[i]:
                error=error+1
    error=error/len(trgt)*100
    return error
        

In [50]:
dataFile=dataReader()
values=makeList(dataFile)
train,test=testSplit(values)
train_lines,train_target=processList(train)
test_lines,test_target=processList(train)
posfreq,negfreq=makeVocab(train_lines,train_target)

In [51]:
posfreq=findProbability(posfreq)
negfreq=findProbability(negfreq)

In [52]:
posprob,negprob=findclassProbability(train_target)

In [53]:
pred=makePrediction(test_lines,posfreq,negfreq,posprob,negprob)
error=findError(pred,test_target)
error

64.62442299622325